# Setup

First things first, let's check our working directory.

In [1]:
pwd  # some bash commands will work without additional characters in Jupyter notebooks if automagic is on

'/Users/ballewbj/snakemake_class/snakemake/Genomics_practice'

To build our pipeline, we'll need to make sure we have access to some dependencies first.  You have a few options here:
1. Use the JupyterLab binder button on the [github repo](https://github.com/marskar/snakemake/tree/master).  All of the depenencies are included in the environment.yml file used to build the binder container.
2. Install dependencies locally:
    - Clone the repo (`git clone https://github.com/marskar/snakemake.git`)
    - Use conda to install the dependencies locally (`conda install -c bioconda <tool=version>`)
3. Create a local environment:
    - Clone the repo (`git clone https://github.com/marskar/snakemake.git`)
    - Use the environment file to build a conda environment locally (`conda env create -f environment.yml`)
    - Be sure to start up `jupyter notebook` from within this environment!

Let's check the versions of the dependencies to ensure they're there and accessible.

In [2]:
# if using option 3 above, check your environment:
!conda env list

# conda environments:
#
base                     /anaconda3
snakemake_class       *  /anaconda3/envs/snakemake_class



In [3]:
# check that we have the required dependencies
!conda list | grep -Ei "snakemake|samtools|picard|bwa|gatk|python"

# packages in environment at /anaconda3/envs/snakemake_class:
bwa                       0.7.17               ha441bb4_5    bioconda
gatk4                     4.1.1.0                       0    bioconda
gitpython                 2.1.11                     py_0    conda-forge
picard                    2.19.0                        0    bioconda
python                    3.6.7             h8dc6b48_1004    conda-forge
python-dateutil           2.8.0                      py_0    conda-forge
python-irodsclient        0.7.0                      py_0    conda-forge
samtools                  1.9                 h7c4ea83_11    bioconda
snakemake                 5.4.5                         0    bioconda
snakemake-minimal         5.4.5                      py_0    bioconda


# Input data

We're going to start with FASTQ files from real human samples, but instead of whole genome data, we're focusing on chromosome 5, position 100000000-101500000.  Our reference genome file also only includes a portion of chr5 (5:99900000-104100000).  This is to keep file sizes small and run times short!  Now, let's make sure we can see the raw files we intend to work on.

In [4]:
ls -lh raw_data/

total 581168
-rw-r--r--  1 ballewbj  NIH\Domain Users    66M Apr 18 11:24 Patient_A.r1.fastq
-rw-r--r--  1 ballewbj  NIH\Domain Users    66M Apr 18 11:24 Patient_A.r2.fastq
-rw-r--r--  1 ballewbj  NIH\Domain Users    76M Apr 18 11:24 Patient_B.r1.fastq
-rw-r--r--  1 ballewbj  NIH\Domain Users    76M Apr 18 11:24 Patient_B.r2.fastq


In [5]:
# what does a fastq look like?
!head -n8 raw_data/Patient_A.r1.fastq  # to run a single line of bash, prepend with "!"

@E00572:97:H5YN2CCXY:5:1101:1773:44327/1
GATGGAGATGAGGAACTTGATGGAAACTGGAGCAAACGTGACTCTTGTTATGCTTTAGCAAAAATACCGGCAGGATTTTGTCCCTGCCCTAGAGATCTGTGGAATTTTGAACTTGAGAGAGAGGATTTAGAGCATCTGCCCCAAGAAAAT
+
<AAFF<JFFFFJJFJJJJ<JJJJFJJJJF-FJJ7FJFFJJJJJFJJJJJJJFJJJJJJ77FFFJJFFFJ<JFFF7-FFJJJFJJJFFFJJJ-7--<<FJJJJ--AA<-<7<F<7F--7A77J-7A-FF-<<A--7F<F--)-)))----7
@E00572:97:H5YN2CCXY:5:1101:1803:49127/1
GCCAAGGGAACCCCCAGCCCTACCCAGGGAAACCGGGAGTGATTGTGTAACTCCAGGAAACCATGCTTCTACCATGGATCTTTGCAACCCATGGATCAGGAGATCCCCCTGTGAGCTCATGCCACCAGGACCTTGGGTCTGACACACAGC
+
AAAAFJJJF<FJ-FJAFFJJJA<FFJAFFFFFFJJFAAJJJJJJFJJJFJJFJJJFJFJJJJJJJJJJJJJJJJJJFFJJJJJFA<7JFAAFJJ-AF7FFFJ-FF-F-AF<J77-A-7F-7FF-AA<JA<A-<<A)-AAF<F<FF--A7)


In [6]:
ls -lh ref_genome/

total 8344
-rw-r--r--  1 ballewbj  NIH\Domain Users   4.1M Apr 18 14:00 chr5_ref.fasta


In [7]:
# what does a reference genome look like?
!head -n4 ref_genome/chr5_ref.fasta

>5
AATAGGAAATCAAAGGGAATTTTAAGAGCTATTTTGAGACAAAAAAAAAATGGCATAACA
AAACTTATGGGATGCAGCAAAAGCATTGCTAGGAGAGAAGTTTATAGCAATAAATGCTTA
TGCTATGAAAGAAGAAAGACTTCAAATAAACAACCTAGCTTTACCCTTTCAGAAAGTGGC


# Plan

__Goal:__ assemble a working DNA-seq pipeline!

- Align sequencing data to a reference genome
- Call variants in the aligned data

# First rule: indexing

Our first rule is going to take our reference genome file, and index it so that the alignment tool can read it.  We can write the rule in any text editor, but for this class, we'll write it here in the notebook and save it to a file.

In [8]:
%%writefile snakefile_test1

ref = 'ref_genome/chr5_ref.fasta'
refNoExt = os.path.splitext(ref)[0]

rule index_ref:
    input:
        ref
    output:
        o1 = ref + '.amb',
        o2 = ref + '.ann',
        o3 = ref + '.bwt',
        o4 = ref + '.pac',
        o5 = ref + '.sa',
        o6 = ref + '.fai',
        o7 = refNoExt + '.dict'
    shell:
        'bwa index {input};'
        'samtools faidx {input};'
        'picard CreateSequenceDictionary REFERENCE={input} OUTPUT={output.o7}'

Writing snakefile_test1


Let's test our first rule!  You can run this rule from the command line or here in the notebook using cell magic.  For our first test, we're going to try a __dry-run__ by using the `-n` flag.  If your snakefile is called something other than "Snakefile," use `-s <filename>`.

In [9]:
!snakemake -ns snakefile_test1

Building DAG of jobs...
Job counts:
	count	jobs
	1	index_ref
	1

[Thu Apr 18 14:20:06 2019]
rule index_ref:
    input: ref_genome/chr5_ref.fasta
    output: ref_genome/chr5_ref.fasta.amb, ref_genome/chr5_ref.fasta.ann, ref_genome/chr5_ref.fasta.bwt, ref_genome/chr5_ref.fasta.pac, ref_genome/chr5_ref.fasta.sa, ref_genome/chr5_ref.fasta.fai, ref_genome/chr5_ref.dict
    jobid: 0

Job counts:
	count	jobs
	1	index_ref
	1
This was a dry-run (flag -n). The order of jobs does not reflect the order of execution.


Looks great!  Note that the dry-run does not actually execute any jobs; it shows the execution plan.  

Now let's try running our pipeline for real.  Add the `-p` flag to print the job that's run for each rule.

In [10]:
!snakemake -ps snakefile_test1

Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	index_ref
	1

[Thu Apr 18 14:20:10 2019]
rule index_ref:
    input: ref_genome/chr5_ref.fasta
    output: ref_genome/chr5_ref.fasta.amb, ref_genome/chr5_ref.fasta.ann, ref_genome/chr5_ref.fasta.bwt, ref_genome/chr5_ref.fasta.pac, ref_genome/chr5_ref.fasta.sa, ref_genome/chr5_ref.fasta.fai, ref_genome/chr5_ref.dict
    jobid: 0

bwa index ref_genome/chr5_ref.fasta;samtools faidx ref_genome/chr5_ref.fasta;picard CreateSequenceDictionary REFERENCE=ref_genome/chr5_ref.fasta OUTPUT=ref_genome/chr5_ref.dict
[bwa_index] Pack FASTA... 0.03 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 0.99 seconds elapse.
[bwa_index] Update BWT... 0.02 sec
[bwa_index] Pack forward-only FASTA... 0.02 sec
[bwa_index] Construct SA from BWT and Occ... 0.32 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index ref_genome/chr5_ref.fasta
[main] Real time:

In [11]:
ls -lh ref_genome

total 22864
-rw-r--r--  1 ballewbj  NIH\Domain Users   161B Apr 18 14:20 chr5_ref.dict
-rw-r--r--  1 ballewbj  NIH\Domain Users   4.1M Apr 18 14:00 chr5_ref.fasta
-rw-r--r--  1 ballewbj  NIH\Domain Users    12B Apr 18 14:20 chr5_ref.fasta.amb
-rw-r--r--  1 ballewbj  NIH\Domain Users    36B Apr 18 14:20 chr5_ref.fasta.ann
-rw-r--r--  1 ballewbj  NIH\Domain Users   4.0M Apr 18 14:20 chr5_ref.fasta.bwt
-rw-r--r--  1 ballewbj  NIH\Domain Users    18B Apr 18 14:20 chr5_ref.fasta.fai
-rw-r--r--  1 ballewbj  NIH\Domain Users   1.0M Apr 18 14:20 chr5_ref.fasta.pac
-rw-r--r--  1 ballewbj  NIH\Domain Users   2.0M Apr 18 14:20 chr5_ref.fasta.sa


Hooray!!  Our first rule worked.  Note that Snakemake stdout provides a beautiful log of steps run and errors encountered.

What happens if we try to run it again?

In [12]:
!snakemake -ps snakefile_test1

Building DAG of jobs...
Nothing to be done.
Complete log: /Users/ballewbj/snakemake_class/snakemake/Genomics_practice/.snakemake/log/2019-04-18T142020.949510.snakemake.log


# Second rule: alignment

Our next rule is going to take the short reads in our FASTQ files, and align them to a reference sequence using a tool called [__bwa__](https://github.com/lh3/bwa).  

In [ ]:
rule align_fastqs: 
    input:
        ref = ref,
        r1 = ref + '.amb',
        r2 = ref + '.ann',
        r3 = ref + '.bwt',
        r4 = ref + '.pac',
        r5 = ref + '.sa',
        fq1 = 'raw_data/Patient_A.r1.fastq',
        fq2 = 'raw_data/Patient_A.r2.fastq'
    output:
        'aligned/PatientA.bam'
    params:
        pl = 'ILLUMINA',
        rg = 'PatientA_rg',
        sm = 'PatientA'
    shell:
        'bwa mem -R "@RG\tID:{params.rg}\tSM:{params.sm}\tPL:{params.pl}" {input.ref} {input.fq1} {input.fq2} | samtools sort -o {output}'

Note how not all the listed input files are actually needed in the command line, but they are required for the command to run successfully (i.e. bwa will give an error if the index files are not there).  Snakemake recommends that you include all file dependencies in the input section, even if they're not used in the command invocation.

This is a good example of the use of `params` in a rule.  Here, they're used to define the metadata for the bam file.

Hold up.  We don't want to hard-code our sample files into a pipeline, or else we have to change the code for every sample! How do we handle this?

![xkcd](https://imgs.xkcd.com/comics/the_general_problem.png)

In [ ]:
SAMPLES = ['Patient_A.r1', 'Patient_A.r2', 'Patient_B.r1', 'Patient_B.r2',]

rule align_fastqs: 
    input:
        ref = ref,
        r1 = ref + '.amb',
        r2 = ref + '.ann',
        r3 = ref + '.bwt',
        r4 = ref + '.pac',
        r5 = ref + '.sa',
        fq = 'raw_data/{sample}.fastq'
    output:
        'aligned/{sample}.bam'
    params:
        pl = 'ILLUMINA',
        rg = '{sample}_rg',
        sm = '{sample}'
    shell:
        'bwa mem -R "@RG\tID:{params.rg}\tSM:{params.sm}\tPL:{params.pl}" {input.ref} {input.fq} | samtools sort -o {output}'

What if we use a list, as above?  This would run an alignment on each fastq individually, which would be fine if we had single-end reads.  But, we have paired-end reads, which means you've sequenced in both directions, and you need to align two related fastq files per sample.

In [ ]:
rawDataPath = 'raw_data/'
sampleDict = {
    'PatientA':['Patient_A.r1.fastq', 'Patient_A.r2.fastq'],
    'PatientB':['Patient_B.r1.fastq', 'Patient_B.r2.fastq']
}

def get_read1_fastq(wildcards):
    (read1, read2) = sampleDict[wildcards.sample]
    return rawDataPath + read1

def get_read2_fastq(wildcards):
    (read1, read2) = sampleDict[wildcards.sample]
    return rawDataPath + read2

rule align_fastqs: 
    input:
        ref = ref,
        r1 = ref + '.amb',
        r2 = ref + '.ann',
        r3 = ref + '.bwt',
        r4 = ref + '.pac',
        r5 = ref + '.sa',
        fq1 = get_read1_fastq,
        fq2 = get_read2_fastq
    output:
        'aligned/{sample}.bam'
    params:
        pl = 'ILLUMINA',
        rg = '{sample}_rg',
        sm = '{sample}'
    shell:
        'bwa mem -R "@RG\tID:{params.rg}\tSM:{params.sm}\tPL:{params.pl}" {input.ref} {input.fq1} {input.fq2} | samtools sort -o {output}'

What if we use a dict?  Much better!  Now our `rule align_fastqs` is generalizable.  If you were using this pipeline in real life, you'd probably require the user to provide a sample file where each line has the sample name, fastq1, and fastq2, and you'd read that in to a dict (rather than explicitly defining a dict like we did).

Note that input (or params) can be the return value of a function, as in this example.

Let's put the two rules together, and then try running them.

In [13]:
%%writefile snakefile_test2

ref = 'ref_genome/chr5_ref.fasta'
refNoExt = os.path.splitext(ref)[0]

rawDataPath = 'raw_data/'
sampleDict = {
    'PatientA':['Patient_A.r1.fastq', 'Patient_A.r2.fastq'],
    'PatientB':['Patient_B.r1.fastq', 'Patient_B.r2.fastq']
}

def get_read1_fastq(wildcards):
    (read1, read2) = sampleDict[wildcards.sample]
    return rawDataPath + read1

def get_read2_fastq(wildcards):
    (read1, read2) = sampleDict[wildcards.sample]
    return rawDataPath + read2


rule index_ref:
    input:
        ref
    output:
        o1 = ref + '.amb',
        o2 = ref + '.ann',
        o3 = ref + '.bwt',
        o4 = ref + '.pac',
        o5 = ref + '.sa',
        o6 = ref + '.fai',
        o7 = refNoExt + '.dict'
    shell:
        'bwa index {input};'
        'samtools faidx {input};'
        'picard CreateSequenceDictionary REFERENCE={input} OUTPUT={output.o7}'

rule align_fastqs: 
    input:
        ref = ref,
        r1 = ref + '.amb',
        r2 = ref + '.ann',
        r3 = ref + '.bwt',
        r4 = ref + '.pac',
        r5 = ref + '.sa',
        fq1 = get_read1_fastq,
        fq2 = get_read2_fastq
    output:
        'aligned/{sample}.bam'
    params:
        pl = 'ILLUMINA',
        rg = '{sample}_rg',
        sm = '{sample}'
    shell:
        'bwa mem -R "@RG\tID:{params.rg}\tSM:{params.sm}\tPL:{params.pl}" {input.ref} {input.fq1} {input.fq2} | samtools sort -o {output}'

Writing snakefile_test2


In [14]:
!snakemake -ps snakefile_test2

Building DAG of jobs...
Nothing to be done.
Complete log: /Users/ballewbj/snakemake_class/snakemake/Genomics_practice/.snakemake/log/2019-04-18T142043.026837.snakemake.log


Oh no!  What went wrong?  We haven't given snakemake a target file.  Let's add a `rule all`.

In [15]:
%%writefile snakefile_test3

ref = 'ref_genome/chr5_ref.fasta'
refNoExt = os.path.splitext(ref)[0]

rawDataPath = 'raw_data/'
sampleDict = {
    'PatientA':['Patient_A.r1.fastq', 'Patient_A.r2.fastq'],
    'PatientB':['Patient_B.r1.fastq', 'Patient_B.r2.fastq']
}

def get_read1_fastq(wildcards):
    (read1, read2) = sampleDict[wildcards.sample]
    return rawDataPath + read1

def get_read2_fastq(wildcards):
    (read1, read2) = sampleDict[wildcards.sample]
    return rawDataPath + read2


rule all:
    input:
        expand('aligned/{sample}.bam', sample=sampleDict.keys())

rule index_ref:
    input:
        ref
    output:
        o1 = ref + '.amb',
        o2 = ref + '.ann',
        o3 = ref + '.bwt',
        o4 = ref + '.pac',
        o5 = ref + '.sa',
        o6 = ref + '.fai',
        o7 = refNoExt + '.dict'
    shell:
        'bwa index {input};'
        'samtools faidx {input};'
        'picard CreateSequenceDictionary REFERENCE={input} OUTPUT={output.o7}'

rule align_fastqs: 
    input:
        ref = ref,
        r1 = ref + '.amb',
        r2 = ref + '.ann',
        r3 = ref + '.bwt',
        r4 = ref + '.pac',
        r5 = ref + '.sa',
        fq1 = get_read1_fastq,
        fq2 = get_read2_fastq
    output:
        'aligned/{sample}.bam'
    params:
        pl = 'ILLUMINA',
        rg = '{sample}_rg',
        sm = '{sample}'
    shell:
        'bwa mem -R "@RG\\tID:{params.rg}\\tSM:{params.sm}\\tPL:{params.pl}" {input.ref} {input.fq1} {input.fq2} | samtools sort -o {output}'

Writing snakefile_test3


Now snakemake knows that `{sample}` should expand to PatientA and PatientB, and that the pipeline should end up producing the files `'aligned/{sample}.bam'`.  Let's try running it (this will take a minute or two):

In [16]:
!snakemake -ps snakefile_test3

Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	2	align_fastqs
	1	all
	3

[Thu Apr 18 14:20:52 2019]
rule align_fastqs:
    input: ref_genome/chr5_ref.fasta, ref_genome/chr5_ref.fasta.amb, ref_genome/chr5_ref.fasta.ann, ref_genome/chr5_ref.fasta.bwt, ref_genome/chr5_ref.fasta.pac, ref_genome/chr5_ref.fasta.sa, raw_data/Patient_B.r1.fastq, raw_data/Patient_B.r2.fastq
    output: aligned/PatientB.bam
    jobid: 2
    wildcards: sample=PatientB

bwa mem -R "@RG\tID:PatientB_rg\tSM:PatientB\tPL:ILLUMINA" ref_genome/chr5_ref.fasta raw_data/Patient_B.r1.fastq raw_data/Patient_B.r2.fastq | samtools sort -o aligned/PatientB.bam
[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 66674 sequences (10000123 bp)...
[M::process] read 66674 sequences (10000097 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (2, 31731, 0, 2)
[M::mem_pestat] skip orientation FF as there are not

[M::mem_process_seqs] Processed 66674 reads in 5.015 CPU sec, 4.921 real sec
[M::process] read 66676 sequences (10000211 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (1, 31694, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (249, 283, 320)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (107, 462)
[M::mem_pestat] mean and std.dev: (285.44, 53.83)
[M::mem_pestat] low and high boundaries for proper pairs: (36, 533)
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] skip orientation RR as there are not enough pairs
[M::mem_process_seqs] Processed 66674 reads in 4.919 CPU sec, 4.829 real sec
[M::process] read 66676 sequences (10000065 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (3, 31685, 0, 0)
[M::mem_pestat] skip orientation FF as there are not en

In [17]:
ls -lh aligned/

total 105544
-rw-r--r--  1 ballewbj  NIH\Domain Users    24M Apr 18 14:22 PatientA.bam
-rw-r--r--  1 ballewbj  NIH\Domain Users    27M Apr 18 14:21 PatientB.bam


Yay!  A few things of note:
- We now have one aligned bam file per patient!
- Snakemake automatically created the directory `aligned/` for us.
- The stdout also includes a bunch of information from bwa.  There are ways to clean this up, but we're going to skip over that for now.
- Snakemake saw that the indexed reference files were already created, so it did not re-run that rule.

# Third rule: index bams

Like the reference genome, the aligned bam files need to be indexed for the next tool to be able to read them.  We'll need to write the rule and update the rule all with the new target file.

In [18]:
%%writefile snakefile_test4

ref = 'ref_genome/chr5_ref.fasta'
refNoExt = os.path.splitext(ref)[0]

rawDataPath = 'raw_data/'
sampleDict = {
    'PatientA':['Patient_A.r1.fastq', 'Patient_A.r2.fastq'],
    'PatientB':['Patient_B.r1.fastq', 'Patient_B.r2.fastq']
}

def get_read1_fastq(wildcards):
    (read1, read2) = sampleDict[wildcards.sample]
    return rawDataPath + read1

def get_read2_fastq(wildcards):
    (read1, read2) = sampleDict[wildcards.sample]
    return rawDataPath + read2


rule all:
    input:
        expand('aligned/{sample}.bam.bai', sample=sampleDict.keys())

rule index_ref:
    input:
        ref
    output:
        o1 = ref + '.amb',
        o2 = ref + '.ann',
        o3 = ref + '.bwt',
        o4 = ref + '.pac',
        o5 = ref + '.sa',
        o6 = ref + '.fai',
        o7 = refNoExt + '.dict'
    shell:
        'bwa index {input};'
        'samtools faidx {input};'
        'picard CreateSequenceDictionary REFERENCE={input} OUTPUT={output.o7}'

rule align_fastqs: 
    input:
        ref = ref,
        r1 = ref + '.amb',
        r2 = ref + '.ann',
        r3 = ref + '.bwt',
        r4 = ref + '.pac',
        r5 = ref + '.sa',
        fq1 = get_read1_fastq,
        fq2 = get_read2_fastq
    output:
        'aligned/{sample}.bam'
    params:
        pl = 'ILLUMINA',
        rg = '{sample}_rg',
        sm = '{sample}'
    shell:
        'bwa mem -R "@RG\\tID:{params.rg}\\tSM:{params.sm}\\tPL:{params.pl}" {input.ref} {input.fq1} {input.fq2} | samtools sort -o {output}'
        
rule index_bams:
    input:
        'aligned/{sample}.bam'
    output:
        'aligned/{sample}.bam.bai'
    shell:
        'samtools index {input}'

Writing snakefile_test4


In [19]:
!snakemake -ps snakefile_test4

Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	all
	2	index_bams
	3

[Thu Apr 18 14:22:14 2019]
rule index_bams:
    input: aligned/PatientB.bam
    output: aligned/PatientB.bam.bai
    jobid: 2
    wildcards: sample=PatientB

samtools index aligned/PatientB.bam
[Thu Apr 18 14:22:14 2019]
Finished job 2.
1 of 3 steps (33%) done

[Thu Apr 18 14:22:14 2019]
rule index_bams:
    input: aligned/PatientA.bam
    output: aligned/PatientA.bam.bai
    jobid: 1
    wildcards: sample=PatientA

samtools index aligned/PatientA.bam
[Thu Apr 18 14:22:14 2019]
Finished job 1.
2 of 3 steps (67%) done

[Thu Apr 18 14:22:14 2019]
localrule all:
    input: aligned/PatientA.bam.bai, aligned/PatientB.bam.bai
    jobid: 0

[Thu Apr 18 14:22:14 2019]
Finished job 0.
3 of 3 steps (100%) done
Complete log: /Users/ballewbj/snakemake_class/snakemake/Genomics_practice/.snakemake/log/2019-04-18T142214.136216.snakemake.log

In [20]:
ls -lh aligned/

total 105576
-rw-r--r--  1 ballewbj  NIH\Domain Users    24M Apr 18 14:22 PatientA.bam
-rw-r--r--  1 ballewbj  NIH\Domain Users   6.4K Apr 18 14:22 PatientA.bam.bai
-rw-r--r--  1 ballewbj  NIH\Domain Users    27M Apr 18 14:21 PatientB.bam
-rw-r--r--  1 ballewbj  NIH\Domain Users   6.5K Apr 18 14:22 PatientB.bam.bai


# Fourth rule: calling

This fourth rule will compare our aligned sequences to the reference genome, look at places where there's a discrepancy, and report back those variants.  We'll need to write the rule and update the rule all with the new target file.

In [21]:
%%writefile snakefile_test5

ref = 'ref_genome/chr5_ref.fasta'
refNoExt = os.path.splitext(ref)[0]

rawDataPath = 'raw_data/'
sampleDict = {
    'PatientA':['Patient_A.r1.fastq', 'Patient_A.r2.fastq'],
    'PatientB':['Patient_B.r1.fastq', 'Patient_B.r2.fastq']
}

def get_read1_fastq(wildcards):
    (read1, read2) = sampleDict[wildcards.sample]
    return rawDataPath + read1

def get_read2_fastq(wildcards):
    (read1, read2) = sampleDict[wildcards.sample]
    return rawDataPath + read2


rule all:
    input:
        expand('called/{sample}.vcf', sample=sampleDict.keys())

rule index_ref:
    input:
        ref
    output:
        o1 = ref + '.amb',
        o2 = ref + '.ann',
        o3 = ref + '.bwt',
        o4 = ref + '.pac',
        o5 = ref + '.sa',
        o6 = ref + '.fai',
        o7 = refNoExt + '.dict'
    shell:
        'bwa index {input};'
        'samtools faidx {input};'
        'picard CreateSequenceDictionary REFERENCE={input} OUTPUT={output.o7}'

rule align_fastqs: 
    input:
        ref = ref,
        r1 = ref + '.amb',
        r2 = ref + '.ann',
        r3 = ref + '.bwt',
        r4 = ref + '.pac',
        r5 = ref + '.sa',
        fq1 = get_read1_fastq,
        fq2 = get_read2_fastq
    output:
        'aligned/{sample}.bam'
    params:
        pl = 'ILLUMINA',
        rg = '{sample}_rg',
        sm = '{sample}'
    shell:
        'bwa mem -R "@RG\\tID:{params.rg}\\tSM:{params.sm}\\tPL:{params.pl}" {input.ref} {input.fq1} {input.fq2} | samtools sort -o {output}'
        
rule index_bams:
    input:
        'aligned/{sample}.bam'
    output:
        'aligned/{sample}.bam.bai'
    shell:
        'samtools index {input}'
        
rule call_variants:
    input:
        ref = ref,
        r1 = ref + '.fai',
        r2 = refNoExt + '.dict',
        bam = 'aligned/{sample}.bam',
        bai = 'aligned/{sample}.bam.bai'
    output:
        'called/{sample}.vcf'
    shell:
        'gatk HaplotypeCaller -I {input.bam} -O {output} -R {input.ref}'

Writing snakefile_test5


In [22]:
!snakemake -ps snakefile_test5

Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	all
	2	call_variants
	3

[Thu Apr 18 14:22:42 2019]
rule call_variants:
    input: ref_genome/chr5_ref.fasta, ref_genome/chr5_ref.fasta.fai, ref_genome/chr5_ref.dict, aligned/PatientB.bam, aligned/PatientB.bam.bai
    output: called/PatientB.vcf
    jobid: 2
    wildcards: sample=PatientB

gatk HaplotypeCaller -I aligned/PatientB.bam -O called/PatientB.vcf -R ref_genome/chr5_ref.fasta
Using GATK jar /anaconda3/envs/snakemake_class/share/gatk4-4.1.1.0-0/gatk-package-4.1.1.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /anaconda3/envs/snakemake_class/share/gatk4-4.1.1.0-0/gatk-package-4.1.1.0-local.jar HaplotypeCaller -I aligned/PatientB.bam -O called/PatientB.vcf -R ref_genome/chr5_ref.fasta
14:22:45.224 INF

14:23:30.426 INFO  NativeLibraryLoader - Loading libgkl_compression.dylib from jar:file:/anaconda3/envs/snakemake_class/share/gatk4-4.1.1.0-0/gatk-package-4.1.1.0-local.jar!/com/intel/gkl/native/libgkl_compression.dylib
Apr 18, 2019 2:23:32 PM shaded.cloud_nio.com.google.auth.oauth2.ComputeEngineCredentials runningOnComputeEngine
INFO: Failed to detect whether we are running on Google Compute Engine.
14:23:32.128 INFO  HaplotypeCaller - ------------------------------------------------------------
14:23:32.129 INFO  HaplotypeCaller - The Genome Analysis Toolkit (GATK) v4.1.1.0
14:23:32.129 INFO  HaplotypeCaller - For support and documentation go to https://software.broadinstitute.org/gatk/
14:23:32.129 INFO  HaplotypeCaller - Executing as ballewbj@NCI-02034622-ML on Mac OS X v10.13.6 x86_64
14:23:32.129 INFO  HaplotypeCaller - Java runtime: OpenJDK 64-Bit Server VM v1.8.0_192-b01
14:23:32.129 INFO  HaplotypeCaller - Start Date/Time: April 18, 2019 2:23:30 PM EDT
14:23:32.129 INFO  Haplo

In [23]:
ls -lh called/

total 1824
-rw-r--r--  1 ballewbj  NIH\Domain Users   423K Apr 18 14:24 PatientA.vcf
-rw-r--r--  1 ballewbj  NIH\Domain Users   4.1K Apr 18 14:24 PatientA.vcf.idx
-rw-r--r--  1 ballewbj  NIH\Domain Users   413K Apr 18 14:23 PatientB.vcf
-rw-r--r--  1 ballewbj  NIH\Domain Users   4.1K Apr 18 14:23 PatientB.vcf.idx


In [24]:
!grep -A5 "^#CHROM" called/PatientA.vcf

#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	PatientA
5	100327	.	G	C	1496.03	.	AC=2;AF=1.00;AN=2;DP=38;ExcessHet=3.0103;FS=0.000;MLEAC=2;MLEAF=1.00;MQ=60.00;QD=25.36;SOR=0.798	GT:AD:DP:GQ:PL	1/1:0,38:38:99:1510,114,0
5	100514	.	A	G	1864.03	.	AC=2;AF=1.00;AN=2;DP=45;ExcessHet=3.0103;FS=0.000;MLEAC=2;MLEAF=1.00;MQ=60.00;QD=28.73;SOR=0.929	GT:AD:DP:GQ:PL	1/1:0,45:45:99:1878,135,0
5	101444	.	T	TTTTA	1380.06	.	AC=2;AF=1.00;AN=2;DP=39;ExcessHet=3.0103;FS=0.000;MLEAC=2;MLEAF=1.00;MQ=60.00;QD=30.97;SOR=1.214	GT:AD:DP:GQ:PL	1/1:0,31:31:93:1394,93,0
5	101497	.	C	T	1283.03	.	AC=2;AF=1.00;AN=2;DP=33;ExcessHet=3.0103;FS=0.000;MLEAC=2;MLEAF=1.00;MQ=60.00;QD=27.24;SOR=1.022	GT:AD:DP:GQ:PL	1/1:0,33:33:99:1297,99,0
5	101748	.	T	C	1395.03	.	AC=2;AF=1.00;AN=2;DP=37;ExcessHet=3.0103;FS=0.000;MLEAC=2;MLEAF=1.00;MQ=60.00;QD=28.20;SOR=0.746	GT:AD:DP:GQ:PL	1/1:0,37:37:99:1409,111,0


# Put it all together

Add some comments and save the finalized pipeline.

In [25]:
%%writefile Genomics_pipeline

# AUTHOR: BB

'''
This pipeline goes from fastq to vcf
for paired-end germline data, and
requires a reference fasta.
'''

# get user data:
ref = 'ref_genome/chr5_ref.fasta'
refNoExt = os.path.splitext(ref)[0]

rawDataPath = 'raw_data/'
sampleDict = {
    'PatientA':['Patient_A.r1.fastq', 'Patient_A.r2.fastq'],
    'PatientB':['Patient_B.r1.fastq', 'Patient_B.r2.fastq']
}

def get_read1_fastq(wildcards):
    (read1, read2) = sampleDict[wildcards.sample]
    return rawDataPath + read1

def get_read2_fastq(wildcards):
    (read1, read2) = sampleDict[wildcards.sample]
    return rawDataPath + read2


rule all:
    input:
        expand('called/{sample}.vcf', sample=sampleDict.keys())

rule index_ref:
    input:
        ref
    output:
        o1 = ref + '.amb',
        o2 = ref + '.ann',
        o3 = ref + '.bwt',
        o4 = ref + '.pac',
        o5 = ref + '.sa',
        o6 = ref + '.fai',
        o7 = refNoExt + '.dict'
    shell:
        'bwa index {input};'
        'samtools faidx {input};'
        'picard CreateSequenceDictionary REFERENCE={input} OUTPUT={output.o7}'

rule align_fastqs: 
    '''
    Align paired-end reads to the indexed
    reference genome.  Create metadata.
    
    bwa complains if using literal tabs, so
    make sure your snakemake command prints
    "\t".
    '''
    input:
        ref = ref,
        r1 = ref + '.amb',
        r2 = ref + '.ann',
        r3 = ref + '.bwt',
        r4 = ref + '.pac',
        r5 = ref + '.sa',
        fq1 = get_read1_fastq,
        fq2 = get_read2_fastq
    output:
        'aligned/{sample}.bam'
    params:
        pl = 'ILLUMINA',
        rg = '{sample}_rg',
        sm = '{sample}'
    shell:
        'bwa mem -R "@RG\\tID:{params.rg}\\tSM:{params.sm}\\tPL:{params.pl}" {input.ref} {input.fq1} {input.fq2} | samtools sort -o {output}'
        
rule index_bams:
    '''
    Index aligned bams.
    '''
    input:
        'aligned/{sample}.bam'
    output:
        'aligned/{sample}.bam.bai'
    shell:
        'samtools index {input}'
        
rule call_variants:
    '''
    Call variants from aligned bams
    using GATK HaplotypeCaller.
    '''
    input:
        ref = ref,
        r1 = ref + '.fai',
        r2 = refNoExt + '.dict',
        bam = 'aligned/{sample}.bam',
        bai = 'aligned/{sample}.bam.bai'
    output:
        'called/{sample}.vcf'
    shell:
        'gatk HaplotypeCaller -I {input.bam} -O {output} -R {input.ref}'

Writing Genomics_pipeline


Let's remove all the files we've generated and run the pipeline as one sequence of rules.

In [26]:
rm -r aligned/ called/ ref_genome/chr5_ref.dict ref_genome/chr5_ref.fasta.*

Now, let's do a dry-run of our complete pipeline.

In [27]:
!snakemake -nps Genomics_pipeline

Building DAG of jobs...
Job counts:
	count	jobs
	2	align_fastqs
	1	all
	2	call_variants
	2	index_bams
	1	index_ref
	8

[Thu Apr 18 14:25:17 2019]
rule index_ref:
    input: ref_genome/chr5_ref.fasta
    output: ref_genome/chr5_ref.fasta.amb, ref_genome/chr5_ref.fasta.ann, ref_genome/chr5_ref.fasta.bwt, ref_genome/chr5_ref.fasta.pac, ref_genome/chr5_ref.fasta.sa, ref_genome/chr5_ref.fasta.fai, ref_genome/chr5_ref.dict
    jobid: 3

bwa index ref_genome/chr5_ref.fasta;samtools faidx ref_genome/chr5_ref.fasta;picard CreateSequenceDictionary REFERENCE=ref_genome/chr5_ref.fasta OUTPUT=ref_genome/chr5_ref.dict

[Thu Apr 18 14:25:17 2019]
rule align_fastqs:
    input: ref_genome/chr5_ref.fasta, ref_genome/chr5_ref.fasta.amb, ref_genome/chr5_ref.fasta.ann, ref_genome/chr5_ref.fasta.bwt, ref_genome/chr5_ref.fasta.pac, ref_genome/chr5_ref.fasta.sa, raw_data/Patient_B.r1.fastq, raw_data/Patient_B.r2.fastq
    output: aligned/PatientB.bam
    jobid: 6
    wildcards: sample=PatientB

bwa mem -R "@R

Great!  Now let's see the DAG (directed acyclic graph).

In [28]:
!snakemake -nps Genomics_pipeline --dag | dot -Tsvg > dag.svg

Building DAG of jobs...


In [29]:
ls

Genomics_pipeline             snakefile_test1
Pipelines_for_genomics.ipynb  snakefile_test2
dag.svg                       snakefile_test3
raw_data/                     snakefile_test4
ref_genome/                   snakefile_test5


![dag](dag.svg)

Looks good - no unexpected recursion or weird relationships.  Now let's run it for real!

In [30]:
!snakemake -ps Genomics_pipeline

Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	2	align_fastqs
	1	all
	2	call_variants
	2	index_bams
	1	index_ref
	8

[Thu Apr 18 14:25:45 2019]
rule index_ref:
    input: ref_genome/chr5_ref.fasta
    output: ref_genome/chr5_ref.fasta.amb, ref_genome/chr5_ref.fasta.ann, ref_genome/chr5_ref.fasta.bwt, ref_genome/chr5_ref.fasta.pac, ref_genome/chr5_ref.fasta.sa, ref_genome/chr5_ref.fasta.fai, ref_genome/chr5_ref.dict
    jobid: 3

bwa index ref_genome/chr5_ref.fasta;samtools faidx ref_genome/chr5_ref.fasta;picard CreateSequenceDictionary REFERENCE=ref_genome/chr5_ref.fasta OUTPUT=ref_genome/chr5_ref.dict
[bwa_index] Pack FASTA... 0.03 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 1.04 seconds elapse.
[bwa_index] Update BWT... 0.02 sec
[bwa_index] Pack forward-only FASTA... 0.02 sec
[bwa_index] Construct SA from BWT and Occ... 0.31 sec
[main] Version: 0.7.17-r1188
[main] CMD:

[M::mem_process_seqs] Processed 58410 reads in 4.047 CPU sec, 3.987 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem -R @RG\tID:PatientB_rg\tSM:PatientB\tPL:ILLUMINA ref_genome/chr5_ref.fasta raw_data/Patient_B.r1.fastq raw_data/Patient_B.r2.fastq
[main] Real time: 33.386 sec; CPU: 33.966 sec
[Thu Apr 18 14:26:23 2019]
Finished job 6.
2 of 8 steps (25%) done

[Thu Apr 18 14:26:23 2019]
rule align_fastqs:
    input: ref_genome/chr5_ref.fasta, ref_genome/chr5_ref.fasta.amb, ref_genome/chr5_ref.fasta.ann, ref_genome/chr5_ref.fasta.bwt, ref_genome/chr5_ref.fasta.pac, ref_genome/chr5_ref.fasta.sa, raw_data/Patient_A.r1.fastq, raw_data/Patient_A.r2.fastq
    output: aligned/PatientA.bam
    jobid: 4
    wildcards: sample=PatientA

bwa mem -R "@RG\tID:PatientA_rg\tSM:PatientA\tPL:ILLUMINA" ref_genome/chr5_ref.fasta raw_data/Patient_A.r1.fastq raw_data/Patient_A.r2.fastq | samtools sort -o aligned/PatientA.bam
[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 66678 sequ

14:26:57.632 INFO  HaplotypeCaller - Done initializing engine
14:26:57.638 INFO  HaplotypeCallerEngine - Disabling physical phasing, which is supported only for reference-model confidence output
14:26:57.649 INFO  NativeLibraryLoader - Loading libgkl_utils.dylib from jar:file:/anaconda3/envs/snakemake_class/share/gatk4-4.1.1.0-0/gatk-package-4.1.1.0-local.jar!/com/intel/gkl/native/libgkl_utils.dylib
14:26:57.651 WARN  NativeLibraryLoader - Unable to find native library: native/libgkl_pairhmm_omp.dylib
14:26:57.651 INFO  PairHMM - OpenMP multi-threaded AVX-accelerated native PairHMM implementation is not supported
14:26:57.651 INFO  NativeLibraryLoader - Loading libgkl_pairhmm.dylib from jar:file:/anaconda3/envs/snakemake_class/share/gatk4-4.1.1.0-0/gatk-package-4.1.1.0-local.jar!/com/intel/gkl/native/libgkl_pairhmm.dylib
14:26:57.669 INFO  IntelPairHmm - Flush-to-zero (FTZ) is enabled when running PairHMM
14:26:57.669 WARN  IntelPairHmm - Ignoring request for 4 threads; not using OpenM

14:27:39.943 INFO  ProgressMeter - Starting traversal
14:27:39.943 INFO  ProgressMeter -        Current Locus  Elapsed Minutes     Regions Processed   Regions/Minute
14:27:49.953 INFO  ProgressMeter -             5:540872              0.2                  2280          13666.3
14:27:59.995 INFO  ProgressMeter -             5:903322              0.3                  4200          12567.3
14:28:10.008 INFO  ProgressMeter -            5:1422749              0.5                  6800          13570.6
14:28:14.057 INFO  HaplotypeCaller - 1021 read(s) filtered by: ((((((((MappingQualityReadFilter AND MappingQualityAvailableReadFilter) AND MappedReadFilter) AND NotSecondaryAlignmentReadFilter) AND NotDuplicateReadFilter) AND PassesVendorQualityCheckReadFilter) AND NonZeroReferenceLengthAlignmentReadFilter) AND GoodCigarReadFilter) AND WellformedReadFilter)
  1021 read(s) filtered by: (((((((MappingQualityReadFilter AND MappingQualityAvailableReadFilter) AND MappedReadFilter) AND NotSecondaryA

In [33]:
# take a peek at the final output files
!for i in called/Patient*vcf; do echo $i; grep -A5 "^#CHROM" $i; echo ""; done

called/PatientA.vcf
#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	PatientA
5	100327	.	G	C	1496.03	.	AC=2;AF=1.00;AN=2;DP=38;ExcessHet=3.0103;FS=0.000;MLEAC=2;MLEAF=1.00;MQ=60.00;QD=25.36;SOR=0.798	GT:AD:DP:GQ:PL	1/1:0,38:38:99:1510,114,0
5	100514	.	A	G	1864.03	.	AC=2;AF=1.00;AN=2;DP=45;ExcessHet=3.0103;FS=0.000;MLEAC=2;MLEAF=1.00;MQ=60.00;QD=28.73;SOR=0.929	GT:AD:DP:GQ:PL	1/1:0,45:45:99:1878,135,0
5	101444	.	T	TTTTA	1380.06	.	AC=2;AF=1.00;AN=2;DP=39;ExcessHet=3.0103;FS=0.000;MLEAC=2;MLEAF=1.00;MQ=60.00;QD=30.97;SOR=1.214	GT:AD:DP:GQ:PL	1/1:0,31:31:93:1394,93,0
5	101497	.	C	T	1283.03	.	AC=2;AF=1.00;AN=2;DP=33;ExcessHet=3.0103;FS=0.000;MLEAC=2;MLEAF=1.00;MQ=60.00;QD=27.24;SOR=1.022	GT:AD:DP:GQ:PL	1/1:0,33:33:99:1297,99,0
5	101748	.	T	C	1395.03	.	AC=2;AF=1.00;AN=2;DP=37;ExcessHet=3.0103;FS=0.000;MLEAC=2;MLEAF=1.00;MQ=60.00;QD=28.20;SOR=0.746	GT:AD:DP:GQ:PL	1/1:0,37:37:99:1409,111,0

called/PatientB.vcf
#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	PatientB
5	100327	.	G	C	641.60	.	AC=1;AF=

In [35]:
# how many variants were called for each patient?
!grep -cv "^#" called/*vcf

called/PatientA.vcf:2284
called/PatientB.vcf:2173
